## PySpark SetUp

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
# install Python Reddit API Wrapper for getting external data Karma
%pip install pyspark==3.2.0
%pip install praw


# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c91c5b0f-209b-4aee-bbcb-1b8adf6b0443;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 416ms :: artifacts dl 29ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


In [4]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import length
import pandas as pd
import praw

## Submission



In [5]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_submissions = f"project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://sagemaker-us-east-1-861795727138/project/submissions/yyyy=*


23/12/07 07:18:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/12/07 07:18:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


shape of the submissions dataframe is 95,932x68
CPU times: user 2.18 s, sys: 200 ms, total: 2.38 s
Wall time: 6min 1s


## Comments 




In [6]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
#s3_path = "s3a://sagemaker-us-east-1-038932893404/project/comments/yyyy=2021/part-00000-90796409-5783-4705-92c0-27c27eda8c4c-c000.snappy.parquet"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-861795727138/project/comments/yyyy=*


shape of the comments dataframe is 1,053,280x21
CPU times: user 454 ms, sys: 67.9 ms, total: 522 ms
Wall time: 9min 23s


In [7]:
# External data: Get posting karma of the creator from Python Reddit API Wrapper
def udf_get_karma(author_str):
    #Reddit API Credential
    client_id = 'UViHJJ217cLodAY9AraMWg'
    client_secret = 'dvpLKVfv7ItyQhHkYPPOK6o_ssI63w'
    user_agent = 'android:com.example.myredditapp:v1.2.3 (by u/sf_suzyfeng)'
    #Reddit Username
    username = 'sf_suzyfeng'
    #Initialize a Reddit instance
    reddit = praw.Reddit(client_id=client_id,client_secret=client_secret,user_agent=user_agent)
    try:
        redditor= reddit.redditor(author_str)
        link_karma= redditor.link_karma
    #if the account was deleted or not found, set Karma to 0 
    except:
        link_karma= 0
    return link_karma

# Creter udf to retrieve Karma 
udf_karma = F.udf(udf_get_karma, IntegerType())

## Top 10 Creators - Sample


In [8]:
# Sampled data part
sampled_submissions = submissions.sample(withReplacement=False, fraction=0.1, seed=123)

# Preparation
sampled_sub_creators= sampled_submissions.select("subreddit","created_utc","author","title", "score", "num_comments")
sampled_sub_creators = sampled_sub_creators.withColumn("score",F.col("score").cast("float"))\
                                         .withColumn("num_comments",F.col("num_comments").cast("float"))\
                                         .withColumn("created_year",F.year(F.to_timestamp(F.col("created_utc").cast('int'))))

# Group submissions data by year and author
sampled_sub_creators= sampled_sub_creators.groupBy("created_year","author").agg(
    F.count("*").alias("sub_count"),
    F.sum("score").alias("total_score"),
    F.sum("num_comments").alias("total_num_cmnt")
).sort('created_year')

# Filter out meaningless data
sampled_sub_creators= sampled_sub_creators.filter((sampled_sub_creators['author']!='[deleted]') &\
                                                                  (sampled_sub_creators['total_score']!=0) &\
                                                                  (sampled_sub_creators['total_num_cmnt']!=0))

# calculate score for the sample data
sampled_sub_creators= sampled_sub_creators.withColumn("creator_score",F.col("total_score")+F.col("total_num_cmnt"))\
                                                          .orderBy("creator_score",ascending= False)

sampled_sub_creators.cache()

# Define the anually window period 
window_prd = Window.partitionBy("created_year").orderBy(F.col("creator_score").desc())

# Filter the first 50 records on creator_score for each year
sampled_sub_creators_window = sampled_sub_creators.withColumn("ranking", F.row_number().over(window_prd)).filter(F.col("ranking") <= 50)

# Exteral data added: posting karma, filter out deleted acound or user not found record 
sampled_sub_creators_window = sampled_sub_creators_window.withColumn("posting_karma", udf_karma(F.col('author'))).filter(F.col('posting_karma') != 0)

# Re-ranking 
sampled_sub_creators_window = sampled_sub_creators_window.withColumn("ranking", F.row_number().over(window_prd))

# Generate sampled top10 creators 
sampled_top10creator = sampled_sub_creators_window.filter(F.col("ranking") <= 10)

# Generate top100 creators, proportional to the duration(2021,2022-12 months, 2023-3 months)
sampled_sub_creators_window = sampled_sub_creators_window.withColumn("top100_reserve",
    F.when(F.col("created_year") == 2021, 45)
    .when(F.col("created_year") == 2022, 45)
    .when(F.col("created_year") == 2023, 10))

# Generate sampled top100 creators 
sampled_top100creator = sampled_sub_creators_window.filter(F.col("ranking") <= F.col("top100_reserve")).drop("top100_reserve")

sampled_top10creator.show()

+------------+--------------------+---------+-----------+--------------+-------------+-------+-------------+
|created_year|              author|sub_count|total_score|total_num_cmnt|creator_score|ranking|posting_karma|
+------------+--------------------+---------+-----------+--------------+-------------+-------+-------------+
|        2021|     CharlieBluebird|        1|    10697.0|         412.0|      11109.0|      1|        16121|
|        2021|   frogandtoadaregay|        3|     8216.0|         204.0|       8420.0|      2|        38390|
|        2021|          cephalized|        1|     8099.0|         165.0|       8264.0|      3|        36554|
|        2021|         uptownxthot|        1|     6620.0|         147.0|       6767.0|      4|        38543|
|        2021|             cigarte|        2|     6089.0|         117.0|       6206.0|      5|       177409|
|        2021|urfavdisappointmentf|        5|     6026.0|         140.0|       6166.0|      6|        61350|
|        2021|     

## Top 10 Creators - Big data


In [9]:
#Big data part

# Preparation
sub_creators= submissions.select("subreddit","created_utc","author","title", "score", "num_comments")
sub_creators = sub_creators.withColumn("score",F.col("score").cast("float"))\
                                         .withColumn("num_comments",F.col("num_comments").cast("float"))\
                                         .withColumn("created_year",F.year(F.to_timestamp(F.col("created_utc").cast('int'))))

# Group submissions data by year and author
sub_creators= sub_creators.groupBy("created_year","author").agg(
    F.count("*").alias("sub_count"),
    F.round(F.avg("score"),2).alias("avg_score"),
    F.round(F.avg("num_comments"),2).alias("avg_num_cmnt")
).sort('created_year')

# Filter out meaningless data
sub_creators= sub_creators.filter((sub_creators['author']!='[deleted]') &\
                                                                  (sub_creators['avg_score']!=0) &\
                                                                  (sub_creators['avg_num_cmnt']!=0))

# Designed for the big data: more accurate ranking
# Observe the sample output, scale up the emphasize on the submisison count of the creators
sub_creators = sub_creators.withColumn("creator_score",F.round((F.col("sub_count") ** 1.1) * (F.col("avg_score") + F.col("avg_num_cmnt")), 3)
).orderBy("creator_score", ascending=False)

sub_creators.cache()

# Define the anually window period 
window_prd = Window.partitionBy("created_year").orderBy(F.col("creator_score").desc())

# Filter the first 60 records on creator_score for each year
sub_creators_window = sub_creators.withColumn("ranking", F.row_number().over(window_prd)).filter(F.col("ranking") <= 60)

# Exteral data added: posting karma, filter out deleted acound or user not found record 
sub_creators_window = sub_creators_window.withColumn("posting_karma", udf_karma(F.col('author'))).filter(F.col('posting_karma') != 0)

# Re-ranking 
sub_creators_window = sub_creators_window.withColumn("ranking", F.row_number().over(window_prd))

# Generate sampled top10 creators 
top10creator = sub_creators_window.filter(F.col("ranking") <= 10).toPandas()

# Generate top100 creators, proportional to the duration(2021,2022-12 months, 2023-3 months)
# Observe the sample output, some authors appear repeatedly, we select more to ensure that there are at least 100 authors
sub_creators_window = sub_creators_window.withColumn("top100_reserve",
    F.when(F.col("created_year") == 2021, 50)
    .when(F.col("created_year") == 2022, 50)
    .when(F.col("created_year") == 2023, 15))

# Generate sampled top100 creators 
top100creator = sub_creators_window.filter(F.col("ranking") <= F.col("top100_reserve")).drop("top100_reserve").toPandas()

#save data
top10creator.to_csv('../../../data/csv/top10creator.csv',index= False)
top100creator.to_csv('../../../data/csv/top100creator.csv', index= False)

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Increase the maximum number of rows and columns displayed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Load the entire csv file
df = pd.read_csv('../../../data/csv/top10creator.csv') 

# Assuming 'Year' is filled down for each group and 'NaN' represents a continuation of the last non-NaN year
df['created year'] = df['created_year'].ffill()

# Now, let's remove any rows that are entirely NaN, which might be separators in your Excel file
df = df.dropna(how='all')

# If the footer contains non-NaN data but you still want to remove it, you could do something like this:

footer_start_text = "Karma(exteral data)*:"
df = df[~df.iloc[:, 0].astype(str).str.startswith(footer_start_text)]

# Convert columns to appropriate data types

df['sub count'] = df['sub_count'].fillna(0).astype(int)
df['avg score'] = df['avg_score'].fillna(0).astype(float)
df['avg num cmnt'] = df['avg_num_cmnt'].fillna(0).astype(float)
df['creator score'] = df['creator_score'].fillna(0).astype(float)
df['posting karma'] = df['posting_karma'].fillna(0).astype(float)

# Drop the original columns with underscores
df = df.drop(columns=['created_year','sub_count', 'avg_score', 'avg_num_cmnt', 'creator_score'])

# Ensure there are no NaN values in the DataFrame before styling
df = df.dropna()

# Now, rearrange the columns
column_order = ['created year', 'author', 'sub count', 'avg score', 'avg num cmnt', 'creator score', 'posting karma']
df = df[column_order]


# Set a style to the DataFrame for better visualization
styled_df = df.style \
    .background_gradient(cmap='BuGn', subset=['sub count', 'avg score', 'avg num cmnt', 'creator score', 'posting karma']) \
    .bar(subset=['sub count', 'avg score', 'avg num cmnt', 'creator score', 'posting karma'], color='#5fba7d') \
    .format({'sub count': "{:,}", 'avg score': "{:.2f}", 'avg num cmnt': "{:.2f}", 'creator score': "{:.2f}", 'posting karma': "{:.2f}"}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', 'lightyellow'), ('color', 'black'), ('font-weight', 'bold')]
    }, {
        'selector': 'td',
        'props': [('text-align', 'center'), ('color', 'black')]
    }, {
        'selector': 'tr:hover',
        'props': [('background-color', '#ffff99'), ('color', 'black')]
    }, { 
        'selector': 'caption',
        'props': [('font-size', '20px'), ('text-align', 'center'), ('color', 'black')]
    }]) \
    .set_caption("Top 10 Creators in MakeupAddiction Subreddit") \
    .hide_index()


# Display the styled DataFrame
display(styled_df)

/tmp/ipykernel_2588/2508370948.py:61: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  .hide_index()


created year,author,sub count,avg score,avg num cmnt,creator score,posting karma
2021,ahbagelxo,121,570.25,24.03,116159.58,129078.00
2021,cigarte,17,3911.88,62.06,89684.03,177409.00
2021,urfavdisappointmentf,38,1174.50,32.87,66008.88,61350.00
2021,sadperson123456,66,602.71,25.85,63073.23,92361.00
2021,Veeeeezy,15,3027.47,88.93,61284.91,325270.00
2021,kyrazyme,21,1966.81,49.52,57412.02,57773.00
2021,glowymel,100,293.73,9.26,48020.68,99984.00
2021,waterfromthevalley,62,444.60,18.05,43339.45,145266.00
2021,Theotherdaytho,17,1823.18,54.65,42378.94,32888.00
2021,Dominorepulsa,22,1241.68,69.55,39295.59,55993.00


In [11]:
# save to html
html_output_path = '../../../data/plots/eda/styled_top_10_creators_dataframe.html'
with open(html_output_path, 'w') as f:
    f.write(styled_df.render())

/tmp/ipykernel_2588/1090427978.py:4: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(styled_df.render())


### Observations:
1. **Creator Score Analysis:**
   - **Definition:** Creator score is generated from the average score and the average number of comments, with an emphasis on the number of submissions of a creator.
   - **High Creator Scores:** Creators like "Ralucamakeup" (2022) and "ahbagelxo" (2022 and 2021) have exceptionally high creator scores, indicating a large following, high average scores, and a significant average number of comments.
2. **Temporal Discrepancy:**
   - There is a notable discrepancy in the data coverage between 2021-2022 (12 months) and 2023 (January to March).
   - When comparing the data across years, there is a general trend of a decrease in the number of submissions and creator scores in 2023 as compared to 2021 and 2022.
3. **External Data (Reddit Karma):**
   - Reddit Karma is considered a reflection of a Redditor's impact on the community.
   - The top 10 creators from 2021 to 2023 consistently exhibit high Karma scores, emphasizing their favorable reception and positive impact on the Reddit community.
4. **User-Specific Insights:**
   - Notable creators like "ahbagelxo," "sadperson123456," and "The_Alchemyst" appear consistently across different years.
   - Interesting fact about creator "Bxx_Fxxking_Smurfs", he is usually not an active user in MakeupAddition, but he asked for suggestions on how to conceal a facial scar in 2022, which garnered significant interaction from the community.

## New Varaible

In [16]:
# Retrieve top creators
topcreators= pd.read_csv('../../../data/csv/top100creator.csv')
topcreators= topcreators['author'].tolist()

# Preparation
comments_var= comments.withColumn("created_hour",F.hour(F.to_timestamp(F.col("created_utc").cast('int'))))\
                                      .withColumn('body_length',F.length(F.col('body')))

submissions_var= submissions.withColumn("created_hour",F.hour(F.to_timestamp(F.col("created_utc").cast('int'))))\
                                            .withColumn('text_length',F.length(F.col('selftext')))\
                                            .withColumn("title_length",F.length(F.col("title")))

# New varaible
submissions_var= submissions_var.withColumn("is_top_100_creator", F.col("author").isin(topcreators))\
                                            .withColumn("is_peak_hour", F.col("created_hour").isin([16, 17, 18, 19, 20, 21, 22, 23, 0]))\
                                            .withColumn("has_media", F.col("media").isNotNull() | F.col("media_embed").getField('content').isNotNull())\
                                            .withColumn("is_long_text",(F.col('text_length') > 200))

comments_var= comments_var.withColumn("is_top_100_creator", F.col("author").isin(topcreators))\
                                      .withColumn("is_long_text",(F.col('body_length') > 100))

# Dummy varaibles(oringinal code)
# submissions_var= submissions_var.withColumn("skincare",F.col("selftext").rlike("""(?i)body|(?i)hair|(?i)facial|(?i)nails|(?i)lip|(?i)sunscreen|(?i)SPF|(?i)acne|(?i)pimples|(?i)scar|(?i)aging"""))\
#                                   .withColumn("skincare_product",F.col("selftext").rlike("""(?i)moisturizer|(?i)cleanser|(?i)serum|(?i)toner|(?i)lotion"""))\
#                                   .withColumn("skincare_product_brand",F.col("selftext").rlike("""(?i)Clinique|(?i)Neutrogena|(?i)Cetaphil|(?i)Kiehl's|(?i)Olay"""))\
#                                   .withColumn("makeup",F.col("selftext").rlike("""(?i)beauty|(?i)bodypaint|(?i)cosmetics|(?i)style|(?i)artist|(?i)cosplay|(?i)fashion|(?i)celebrity|(?i)party|(?i)wedding|(?i)palette"""))\
#                                   .withColumn("makeup_product",F.col("selftext").rlike("""(?i)eyeliner|(?i)contour|(?i)foundation|(?i)blush|(?i)lipstick|(?i)concealer"""))\
#                                   .withColumn("makeup_product_brand",F.col("selftext").rlike("""(?i)MAC|(?i)NARS|(?i)Sephora|(?i)Fenty|(?i)Revlon|(?i)NYX|(?i)L'Oreal|(?i)Maybelline"""))
# comments_var= comments_var.withColumn("skincare",F.col("body").rlike("""(?i)body|(?i)hair|(?i)facial|(?i)nails|(?i)lip|(?i)sunscreen|(?i)SPF|(?i)acne|(?i)pimples|(?i)scar|(?i)aging"""))\
#                                   .withColumn("skincare_product",F.col("body").rlike("""(?i)moisturizer|(?i)cleanser|(?i)serum|(?i)toner|(?i)lotion"""))\
#                                   .withColumn("skincare_product_brand",F.col("body").rlike("""(?i)Clinique|(?i)Neutrogena|(?i)Cetaphil|(?i)Kiehl's|(?i)Olay"""))\
#                                   .withColumn("makeup",F.col("body").rlike("""(?i)beauty|(?i)bodypaint|(?i)cosmetics|(?i)style|(?i)artist|(?i)cosplay|(?i)fashion|(?i)celebrity|(?i)party|(?i)wedding|(?i)palette"""))\
#                                   .withColumn("makeup_product",F.col("body").rlike("""(?i)eyeliner|(?i)contour|(?i)foundation|(?i)blush|(?i)lipstick|(?i)concealer"""))\
#                                   .withColumn("makeup_product_brand",F.col("body").rlike("""(?i)MAC|(?i)NARS|(?i)Sephora|(?i)Fenty|(?i)Revlon|(?i)NYX|(?i)L'Oreal|(?i)Maybelline"""))\


# Dummy varaibles(code improved with assistance from OpenAI's ChatGPT )
from functools import reduce

categories_dict = {
    "skincare": ["skin","skincare","body", "hair", "facial", "nails", "lip", "sunscreen", "SPF", "acne", "pimples", "scar", "aging"],
    "skincare_product": ["moisturizer", "cleanser", "serum", "toner", "lotion","gel","mask","mist", "oil"],
    "skincare_product_brand": ["Clinique", "Neutrogena", "Cetaphil", "Kiehl's", "Olay","La Mer","Origins","Fresh","Shiseido","Avene"],
    "makeup": ["makeup","beauty","Sephora","bodypaint", "cosmetics", "style", "artist", "cosplay", "fashion", "celebrity", "party", "wedding", "palette"],
    "makeup_product": ["eyeliner", "contour", "foundation", "blush", "lipstick", "concealer","palette", "highlighter","powder","spray","blush"],
    "makeup_product_brand": ["MAC", "NARS", "Sephora", "Fenty", "Revlon", "NYX", "L'Oreal", "Maybelline","Urban Decay","Make Up For Ever","ColourPop"]}

def is_dummy(data, column_name, patterns, text_col):
    pattern_expr = "|".join(f"(?i){pattern}" for pattern in patterns)
    return data.withColumn(column_name, F.col(text_col).rlike(pattern_expr))

# Apply the function to create new columns
submissions_var = reduce(lambda df, col_patterns: is_dummy(df, *col_patterns, 'selftext'), categories_dict.items(), submissions_var)
comments_var = reduce(lambda df, col_patterns: is_dummy(df, *col_patterns, 'body'), categories_dict.items(), comments_var)
# ChatGPT code end 

comments_var.cache()
submissions_var.cache()

# Column names to analyze
sub_columns = ['is_top_100_creator', 'is_peak_hour', 'has_media','is_long_text', 'skincare', 'skincare_product','skincare_product_brand','makeup','makeup_product','makeup_product_brand']
cmt_columns = ['is_top_100_creator', 'is_long_text', 'skincare', 'skincare_product','skincare_product_brand','makeup','makeup_product','makeup_product_brand']

# Submissions group by and count
submissions_var_count = pd.concat(
    [submissions_var.groupby(col).count().withColumn(col, F.col(col).cast('string')).toPandas() for col in sub_columns],
    axis=1)

# Comments group by and count
comments_var_count = pd.concat(
    [comments_var.groupby(col).count().withColumn(col, F.col(col).cast('string')).toPandas() for col in cmt_columns],
    axis=1)

23/12/07 12:27:43 WARN CacheManager: Asked to cache already cached data.
23/12/07 12:27:43 WARN CacheManager: Asked to cache already cached data.


In [ ]:
submissions_var_count

In [ ]:
comments_var_count